In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.appName('titanic').getOrCreate()

In [2]:
df = spark.read.format('csv').load('/FileStore/tables/Machine_learning/titanic.csv', inferschema = True, header = True)

In [3]:
#df.show()
#df.printSchema()
df.columns

In [4]:
my_cols = df.select(['Survived',
 'Pclass','Sex',
 'Age',
 'SibSp',
 'Parch','Fare', 'Embarked'])

In [5]:
#my_cols.show()
my_final_data = my_cols.na.drop()

In [6]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol= 'SexIndex')

In [7]:
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [8]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol= 'EmbIndex')
embark_encoder = OneHotEncoder(inputCol='EmbIndex', outputCol='EmbVec')

In [9]:
assembler = VectorAssembler (inputCols= ['Pclass', 'SexVec', 'EmbVec', 'Age', 'SibSp', 'Parch', 'Fare'], outputCol='features')

In [10]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [11]:
pipeline = Pipeline(stages =[gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg_titanic])

In [12]:
train_Set, test_set = my_final_data.randomSplit([0.7, 0.3])

In [13]:
fit_model = pipeline.fit(train_Set)

In [14]:
results = fit_model.transform(test_set)

In [15]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction', labelCol ='Survived')

In [16]:
results.select('Survived', 'prediction').show()

In [17]:
AUC = my_eval.evaluate(results)
AUC